In [1]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import SABRModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
# -------
# INPUTS
# -------

# Pricing parameters
K     = 100
alpha = 0.25
beta  = 1.0
rho   = 0.0
nu    = 0.0001
r     = 0
slide_to_compute = [-0.3, 0.3]

In [17]:
market_data_list = list()
for bd in range(1, 20):
    T = bd / 250
    for F in np.linspace(start=1, stop=200, num=199, dtype=int):
        option_type = 'call'
        pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
        market_data_list.append({
            'symbol': f"{bd}T_{K}K_{option_type}",
            'option_type': option_type, 
            'time_to_maturity': bd, 
            'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**pricing_results})

        option_type = 'put'
        pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
        market_data_list.append({
            'symbol': f"{bd}T_{K}K_{option_type}",
            'option_type': option_type, 
            'time_to_maturity': bd, 
            'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**pricing_results})
market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

In [29]:
market_data_df[(market_data_df['F']==100) & (market_data_df['time_to_maturity']==19)]

,option_type,time_to_maturity,F,r,K,T,K/F,IV,price,delta,gamma,vega,theta,vanna,volga,-0.30,0.30
symbol,,,,,,,,,,,,,,,,,
19T_100K_call,call,19,100,0,100,0.08,0.00,0.25,2.75,0.51,0.06,0.11,-0.07,0.05,-0.05,12.66,11.84
19T_100K_put,put,19,100,0,100,0.08,0.00,0.25,2.75,-0.49,0.06,0.11,-0.07,0.05,-0.05,12.66,11.84


In [35]:
market_data_df[(market_data_df['F']==80) & (market_data_df['time_to_maturity']==19)]

,option_type,time_to_maturity,F,r,K,T,K/F,IV,price,delta,gamma,vega,theta,vanna,volga,-0.30,0.30
symbol,,,,,,,,,,,,,,,,,
19T_100K_call,call,19,80,0,100,0.08,0.25,0.25,0.00,0.00,0.00,0.00,-0.00,0.03,2.18,0.02,5.23
19T_100K_put,put,19,80,0,100,0.08,0.25,0.25,20.00,-1.00,0.00,0.00,-0.00,0.03,2.18,0.02,5.23


In [37]:
market_data_df[(market_data_df['F']==120) & (market_data_df['time_to_maturity']==19)]

,option_type,time_to_maturity,F,r,K,T,K/F,IV,price,delta,gamma,vega,theta,vanna,volga,-0.30,0.30
symbol,,,,,,,,,,,,,,,,,
19T_100K_call,call,19,120,0,100,0.08,-0.17,0.25,20.01,1.00,0.00,0.00,-0.00,-0.11,10.19,15.87,0.12
19T_100K_put,put,19,120,0,100,0.08,-0.17,0.25,0.01,-0.00,0.00,0.00,-0.00,-0.11,10.19,15.87,0.12


In [ ]:
# Scenario parameters
num = int((scenario_max-scenario_min)/0.01 + 1)
slide_to_compute = list(np.round(np.linspace(start=scenario_min, stop=scenario_max, num=num), 2))

# Option portfolio
market_data_list = list()

# Option ATM
K = F
T = atm_bd / 250
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{atm_bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': atm_bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

for strike in option_strikes:
    K = F * (1+strike)
    T = tail_bd / 250
    option_type = 'call' if K >= F else 'put'
    market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
    market_data_list.append({
        'symbol': f"{tail_bd}T_{K}K_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': tail_bd, 
        'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

In [ ]:
# ATM Scenarios
atm_market_data = market_data_df.loc[market_data_df['F']==market_data_df['K']]
atm_qty = -50000 / atm_market_data['theta']
atm_scenario = atm_market_data.loc[:, ['theta'] + slide_to_compute]
atm_scenario = atm_scenario.drop(0, axis=1)
atm_scenario.rename(columns={'theta': 0}, inplace=True)
atm_scenario = atm_scenario.sort_index(axis=1)
atm_scenario = atm_scenario.multiply(atm_qty, axis=0)
atm_scenario = atm_scenario.iloc[0].transpose()
target_hedge_portfolio_scenario = atm_scenario*-1

In [ ]:
# OTM Scenarios
otm_market_data = market_data_df.loc[market_data_df['F']!=market_data_df['K']]
otm_scenario = otm_market_data.loc[:, ['theta'] + slide_to_compute]
otm_scenario = otm_scenario.drop(0, axis=1)
otm_scenario.rename(columns={'theta': 0}, inplace=True)
otm_scenario = otm_scenario.sort_index(axis=1)
otm_scenario = otm_scenario.transpose()

In [ ]:
X = otm_scenario.loc[flat_scenarios].values
b = target_hedge_portfolio_scenario.loc[flat_scenarios].values
w = np.linalg.solve(X, b)
weights = pd.Series(w, index=otm_market_data.index)
print(weights)

In [ ]:
total_scenario = atm_scenario + (otm_scenario * weights).sum(axis=1)

In [ ]:
total_scenario = total_scenario*(-50000/total_scenario.loc[0])

In [ ]:
test = atm_scenario-total_scenario

In [ ]:
test

In [ ]:
px.line(test)

In [ ]:
px.line(total_scenario)

In [ ]:
px.line(atm_scenario)